# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком
Это задание поможет вам лучше разобраться в методах множественной проверки гипотез и позволит применить ваши знания на данных из реального биологического исследования.

#### В этом задании вы:

вспомните, что такое t-критерий Стьюдента и для чего он применяется
сможете применить технику множественной проверки гипотез и увидеть собственными глазами, как она работает на реальных данных
почувствуете разницу в результатах применения различных методов поправки на множественную проверку

## Основные библиотеки и используемые методы:

Библиотека scipy и основные статистические функции:http://docs.scipy.org/doc/scipy/reference/stats.html#statistical-functions

Библиотека statmodels для методов коррекции при множественном сравнении:

http://statsmodels.sourceforge.net/devel/stats.html

Статья, в которой рассматриваются примеры использования statsmodels для множественной проверки гипотез:

http://jpktd.blogspot.ru/2013/04/multiple-testing-p-value-corrections-in.html

## Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).


Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

## Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

$$F_{c}(C,T) = \begin{cases} \frac{T}{C}, T>C \\ -\frac{C}{T}, T<C \end{cases}$$

где C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

## Инструкции к решению задачи

Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.

### Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

 * для групп **normal (control)** и **early neoplasia (treatment)**
 * для групп **early neoplasia (control)** и **cancer (treatment)**

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

def write_answer_to_file(answer, filename):
    with open(filename, 'w') as f_out:
        f_out.write(str(answer))

In [2]:
pd.options.display.max_rows = 100
data = pd.read_csv('data/gene_high_throughput_sequencing.csv')
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


###   Посчитаем вручную t-критерий Стюдента по формуле
![Image](https://wikimedia.org/api/rest_v1/media/math/render/svg/67f95e04c3d8df149a1fd64c6e7126dd073fe135)

In [3]:
# data = data.drop(['Patient_id','Diagnosis'],axis=1)
compare = pd.DataFrame()


compare['normal_X'] = data[data['Diagnosis'].isin(['normal'])].mean()
compare['early_neoplasia_X'] = data[data['Diagnosis'].isin(['early neoplasia'])].mean()
compare['cancer_X'] = data[data['Diagnosis'].isin(['cancer'])].mean()

compare['normal_std'] = data[data['Diagnosis'].isin(['normal'])].std()
compare['early_neoplasia_std'] = data[data['Diagnosis'].isin(['early neoplasia'])].std()
compare['cancer_std'] = data[data['Diagnosis'].isin(['cancer'])].std()
compare['normal-neoplasia_hands_t'] = (compare['normal_X'] - compare['early_neoplasia_X'])/np.sqrt(((compare['normal_std']**2)/24) + ((compare['early_neoplasia_std']**2)/24))

compare.head(3)

,normal_X,early_neoplasia_X,cancer_X,normal_std,early_neoplasia_std,cancer_std,normal-neoplasia_hands_t
LOC643837,2.681277,2.510894,2.186060,1.508486,1.469398,1.257218,0.396372
LOC100130417,4.368497,8.721781,8.190456,3.045372,3.555897,4.484989,-4.555281
SAMD11,15.159566,18.531325,23.692614,6.829965,5.269496,12.648352,-1.914825


In [4]:
# Посчитаем функцией ttest_ind t-критерий Стюдента и p-value (при построении графика указывается полученное
# t-значение и количество элементов 24-1=23)
from scipy.stats import ttest_ind
compare['normal-neoplasia_t'] = ttest_ind(np.array(data[data['Diagnosis'].isin(['normal'])].drop(['Patient_id','Diagnosis'],axis=1)),
         np.array(data[data['Diagnosis'].isin(['early neoplasia'])].drop(['Patient_id','Diagnosis'],axis=1)),
                            equal_var = False)[0]
compare['normal-neoplasia_p'] = ttest_ind(np.array(data[data['Diagnosis'].isin(['normal'])].drop(['Patient_id','Diagnosis'],axis=1)),
         np.array(data[data['Diagnosis'].isin(['early neoplasia'])].drop(['Patient_id','Diagnosis'],axis=1)), 
                               equal_var = False)[1]

compare['neoplasia-cancer_t'] = ttest_ind(np.array(data[data['Diagnosis'].isin(['early neoplasia'])].drop(['Patient_id','Diagnosis'],axis=1)),
         np.array(data[data['Diagnosis'].isin(['cancer'])].drop(['Patient_id','Diagnosis'],axis=1)),
                            equal_var = False)[0]
compare['neoplasia-cancer_p'] = ttest_ind(np.array(data[data['Diagnosis'].isin(['early neoplasia'])].drop(['Patient_id','Diagnosis'],axis=1)),
         np.array(data[data['Diagnosis'].isin(['cancer'])].drop(['Patient_id','Diagnosis'],axis=1)), 
                               equal_var = False)[1]
compare.head(3)

,normal_X,early_neoplasia_X,cancer_X,normal_std,early_neoplasia_std,cancer_std,normal-neoplasia_hands_t,normal-neoplasia_t,normal-neoplasia_p,neoplasia-cancer_t,neoplasia-cancer_p
LOC643837,2.681277,2.510894,2.186060,1.508486,1.469398,1.257218,0.396372,0.400289,0.690766,0.824849,0.413735
LOC100130417,4.368497,8.721781,8.190456,3.045372,3.555897,4.484989,-4.555281,-4.608766,0.000032,0.452236,0.653429
SAMD11,15.159566,18.531325,23.692614,6.829965,5.269496,12.648352,-1.914825,-1.929277,0.060273,-1.817266,0.079556


In [5]:
answer_11 = compare[compare['normal-neoplasia_p'] < 0.05].shape[0]
answer_12 = compare[compare['neoplasia-cancer_p'] < 0.05].shape[0]
print('Количество статистически значимых генов первой группы:',answer_11)
print('Количество статистически значимых генов второй группы:',answer_12)

Количество статистически значимых генов первой группы: 1575
Количество статистически значимых генов второй группы: 3490


In [6]:
write_answer_to_file(answer_11, 'answers/4.1. Statistics. Genome_task_answer_11.txt')
write_answer_to_file(answer_12,'answers/4.1. Statistics. Genome_task_answer_12.txt')

### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

`import statsmodels.stats.multitest as smm`

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5.

**Обратите внимание, что**

применять поправку на множественную проверку нужно ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.
при использовании поправки на уровне значимости 0.025 меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом 0.025, а не 0.05)!



In [7]:
import statsmodels.stats.multitest as smm
def fold_change(x):
    T=x[1]
    C=x[0]
    if T > C:
        return (T/C)
    else:
        return (-C/T)

In [8]:
compare['normal_neoplasia_fc'] = compare.loc[:,['normal_X','early_neoplasia_X']].apply(fold_change, axis=1)
compare['neoplasia_cancer_fc'] = compare.loc[:,['early_neoplasia_X','cancer_X']].apply(fold_change, axis=1)
compare[['normal_X','early_neoplasia_X','cancer_X','normal_neoplasia_fc','neoplasia_cancer_fc']].head(3)

,normal_X,early_neoplasia_X,cancer_X,normal_neoplasia_fc,neoplasia_cancer_fc
LOC643837,2.681277,2.510894,2.186060,-1.067858,-1.148593
LOC100130417,4.368497,8.721781,8.190456,1.996517,-1.064871
SAMD11,15.159566,18.531325,23.692614,1.222418,1.278517


In [9]:
import statsmodels.stats.multitest as smm
compare['normal_neoplasia_fc_smm'] = smm.multipletests(compare['normal-neoplasia_p'], alpha=0.025, method='holm')[0]
compare['neoplasia_cancer_fc_smm'] = smm.multipletests(compare['neoplasia-cancer_p'], alpha=0.025, method='holm')[0]
compare[['normal_X','early_neoplasia_X','cancer_X','normal_neoplasia_fc',
         'neoplasia_cancer_fc','normal_neoplasia_fc_smm','neoplasia_cancer_fc_smm']].head(3)

,normal_X,early_neoplasia_X,cancer_X,normal_neoplasia_fc,neoplasia_cancer_fc,normal_neoplasia_fc_smm,neoplasia_cancer_fc_smm
LOC643837,2.681277,2.510894,2.186060,-1.067858,-1.148593,False,False
LOC100130417,4.368497,8.721781,8.190456,1.996517,-1.064871,False,False
SAMD11,15.159566,18.531325,23.692614,1.222418,1.278517,False,False


In [10]:
answer_21 = compare[(compare['normal_neoplasia_fc_smm'] == True) & (abs(compare['normal_neoplasia_fc']) > 1.5)].shape[0]
answer_22 = compare[(compare['neoplasia_cancer_fc_smm'] == True) & (abs(compare['neoplasia_cancer_fc']) > 1.5)].shape[0]
print('Количество значимых отличий в первой группе после коррекции Холма-Бонферрони:',answer_21)
print('Количество значимых отличий во второй группе после коррекции Холма-Бонферрони:', answer_22)

Количество значимых отличий в первой группе после коррекции Холма-Бонферрони: 2
Количество значимых отличий во второй группе после коррекции Холма-Бонферрони: 77


In [11]:
write_answer_to_file(answer_21, 'answers/4.1. Statistics. Genome_task_answer_21.txt')
write_answer_to_file(answer_22,'answers/4.1. Statistics. Genome_task_answer_22.txt')

### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.



In [12]:
compare['normal_neoplasia_fc_fdr_bh'] = smm.multipletests(compare['normal-neoplasia_p'], alpha=0.025, method='fdr_bh')[0]
compare['neoplasia_cancer_fc_fdr_bh'] = smm.multipletests(compare['neoplasia-cancer_p'], alpha=0.025, method='fdr_bh')[0]
compare[['normal_X','early_neoplasia_X','cancer_X','normal_neoplasia_fc',
         'neoplasia_cancer_fc','normal_neoplasia_fc_fdr_bh','neoplasia_cancer_fc_fdr_bh']].head(3)

,normal_X,early_neoplasia_X,cancer_X,normal_neoplasia_fc,neoplasia_cancer_fc,normal_neoplasia_fc_fdr_bh,neoplasia_cancer_fc_fdr_bh
LOC643837,2.681277,2.510894,2.186060,-1.067858,-1.148593,False,False
LOC100130417,4.368497,8.721781,8.190456,1.996517,-1.064871,False,False
SAMD11,15.159566,18.531325,23.692614,1.222418,1.278517,False,False


In [13]:
answer_31 = compare[(compare['normal_neoplasia_fc_fdr_bh'] == True) & (abs(compare['normal_neoplasia_fc']) > 1.5)].shape[0]
answer_32 = compare[(compare['neoplasia_cancer_fc_fdr_bh'] == True) & (abs(compare['neoplasia_cancer_fc']) > 1.5)].shape[0]
print('Количество значимых отличий в первой группе после коррекции Холма-Бонферрони:', answer_31)
print('Количество значимых отличий во второй группе после коррекции Холма-Бонферрони:', answer_32)

Количество значимых отличий в первой группе после коррекции Холма-Бонферрони: 4
Количество значимых отличий во второй группе после коррекции Холма-Бонферрони: 524


In [14]:
write_answer_to_file(answer_31, 'answers/4.1. Statistics. Genome_task_answer_31.txt')
write_answer_to_file(answer_32,'answers/4.1. Statistics. Genome_task_answer_32.txt')